# Recurrent Neural Networks
### Machine Learning Reading Group,  The University of Melbourne

&nbsp;

Dr Peter Cudmore  
Systems Biology Lab  
The University of Melbourne

&nbsp;

Slides at:  
https://github.com/peter-cudmore/seminars/

## Recurrent Neural Networks

Recurrent Neural Networks (RNN's) are neural networks with'memory'.  

Useful for:
- Natural Language Processing
- Signal Reconstructing
- Motion Tracking/Control

Recurrent Neural Networks good for *sequential data*.

## Feedback in RNN's

Memory is usually implemented by introduction *feedback loops* into the neural network.  

Feedback can be introduced at different places in the network; at the output stage, between layers, or at the individual artifical neuron level.


Introducing feedback has some consequences:

- Topolgy is very important.
- 'Infinite Impulse Response' implies truncated training
- Gradient magnitude issues are very common.
- Stability, strange attractors, chaos, etc...

- Draw an example of a neural network and contrast agianst one with a feedback loop (ie, RNN).
- Draw output feedback topology, layer feedback, node feedback 


## RNNs and Signal Processing


Suppose:
- We have partial and/or noisy observations $x(t)$ of some sequential process 
- which is asumed to evolve on a space $z$ according to some evolution rule $f$,
- and is measured to make a prediciton or desicion $\hat{y}(t)$.

If we were doing control systems, might write this as  

$$
\dot{z} = f(z,x)\qquad \hat{y} = g(z,x)
$$  
to predict $\hat{y}$ from some sequence of values of $x$.

- Useful to refer to linear control systems $\dot{z} = Az + Bx$ and $\hat{y} = Cz + Dx$ 
- $z$ is always going to refer to interal state
- $x$ is always going to be input data
- $\hat{y}$ is always going to be RNN output
- $y$ is always going to be ground truth, when it is know.

## RNNs and Digital Signal Processing
Consider
$$\dot{z} = f(z,x)\qquad \hat{y} = g(z,x).$$

A RNN can be intepreted as:
- Applying some discritization scheme to the evolution on some sufficiently large state space such that $z_n = F(z_{n-1},x_n)$.
- Applying the *Universal Function Theorem*  to approximate the resulting function $F$ with a feed-forward neural network.
- Conceptually discritizing and splitting $g$ into post-processing (for example, soft-max to transform the output layer data into a pdf) and 'everything else' (which then assimmilated into the network)

RNN in general form::
$$
z_n = F(z_{n-1}, x_n; \mathbb{\theta}),\qquad \hat{y}_n = M(z_n, x_n)
$$

Observations:
- Different topolgoes correspond to different discritization schemes
- RNN's are _compositional_ (more on this soon)
- RNN's are Iterated Funciton Systems: class of functions know to generate fractals etc.
- The diff-eq representation makes it clear that we need some $z(0)$ (we usually assume to be zero for training).

## Example RNN
A simple example of a RNN with one hidden layer is
$$
\begin{eqnarray}
z_n &=& b + Wh_{n-1} + Ux_n\\
h_{n} &=& \tanh(z_n)\\
o_n &=& c +Vh_n\\
\hat{y}_n &=& \text{softmax}(o_n)
\end{eqnarray}$$

In this case
$$ z_n = F(z_{n-1}, x_n) = b + W\tanh(z_{n-1}) + Ux_n$$
and

$$ G(z_n,x_n) = \text{softmax}\left(c + V\tanh(z_n)\right)$$

Here:
 - parameters are $\theta = \{b, W, U, c, V\}$,
 - $z_n$ is the state at step $n$,
 - $h_n$ is the hidden layer at step $n$,
 - $o_n$ is the output layer at step $n$,
 - $\tanh$ is the elementwise tanh
 - $\text{softmax}(o_n) = \exp(o_n)/\sum(\exp(o_n))$ is the softmax, or normalised  elementwise exponential.

## Composition and Unfolding 

Instead of thinking of a RNN acting in an iterative sense, it is often useful to 'unfold' the neural network, that is think of it as a map from $G: X^N \rightarrow Y^N$, where $x_1,x_2\in X$ (for example, vectors EEG data at time $t$) and $y_1\in Y$ (the corresponding decision/ouptut). 

Recall:
$$z_n = F(z_{n-1}, x_n; \mathbb{\theta}),\qquad y_n = M(z_n, x_n)$$
then it follows that
$$ \hat{y}_1 = M(F(z_0,x_1), x_1),\quad \hat{y}_2 = M(F(F(z_0,x_1),x_2),x_2), \quad \ldots$$   
Lets define $M_n(z) := M(z,x_n)$ and $F_n:= F(z,x_n)$ then it follows that
$$
\begin{eqnarray}
\hat{y}_1 &=& G_1(z_0, x_1) = M_1\circ F_1(z_0) = M_1(F_1(z_0)) \\
 \hat{y}_2 &=& G_2(z_0, x_1, x_2) = M_2 \circ F_2\circ F_1(z_0) \\
& &\qquad\vdots \\
 \hat{y}_N  &=&G_N(z_0,x_1,x_2,\ldots,x_N;
\mathbb{\theta}) =  M_N\circ F_N\circ F_{N-1}\circ\cdots\circ F_1 (z_0)\end{eqnarray}$$

Draw 
![Unrolled RNN](images/UnrolledRNN.svg)

## Advantages of Unfolding

1. Unfolding makes the input-ouput conditioning explicit
2. Writing out the composition sequences makes it clear how to do gradient descent. 

Applying backprop to the unfolded graph is known as _Back-propogation through time._

## Training A RNN with Backpropogation through time

Requires:
- A sequence size $N$, which determines how far to unroll the graph.
- Test input sequence $\{x_n\}_{n=1}^K$ and corresponding true outputs $\{y_n\}_{n=1}^K$ for some $K = N + k$ where $k$ is the batch size.
- A loss function $L$.
- An initial guess for the state: $z_0 =0$.
- The RNN $F(z,x;\theta)$.
- The graph unfolding $G_N(x_1,x_2,\ldots;\theta)$


Algorithm:

    while training:
        z = 0
        for step from 0 to k:
            test_sequence = x[step:step+N]
            y_hat = G_N(z, test_sequence, theta)
            error = y - y_hat
            theta = backprop(G, L, theta, error). 
            z = F(z, x[step+ N], theta)

- Computation time is $O(N)$
- Memory is $O(N)$
- Can have problems with local mins.

## Example RNN redux
A simple example of a RNN with one hidden layer is
$$
\begin{eqnarray}
z_n &=& b + Wh_{n-1} + Ux_n\\
h_{n} &=& \tanh(z_n)\\
o_n &=& c +Vh_n\\
\hat{y}_n &=& \text{softmax}(o_n)
\end{eqnarray}$$

In this case
$$ z_n = F(z_{n-1}, x_n) = b + W\tanh(z_{n-1}) + Ux_n$$
and

$$ G(z_n,x_n) = \text{softmax}\left(c + V\tanh(z_n)\right)$$

For a sequence of lenght $N$, $\nabla_{o_N}L$ is known.  
We compute
$$\nabla_{h_N} L = V^T \nabla_{o_N}L$$
and for any $0<n<N$ 
$$
\begin{eqnarray}
\nabla_{h_{n-1}}L &=& \left(\frac{\partial h_n}{\partial n_{n-1}}\right)^T(\nabla_{h_n}L) + \left(
\frac{\partial o_{n-1}}{\partial h_{n-1}}\right)^T\nabla_{o_n}L\\
&=& W^T\text{diag}(1-h_n^2)(\nabla_{h_n}L) + V^T \nabla_{o_n}L
\end{eqnarray}
$$

The $n-1$th step can be computed from the $n$th step.  
This relies on the fact that $$
f(x) = \tanh(z) \implies \frac{\partial f}{\partial x} = 1 - \tanh{x}^2 =  1 - f(x)^2 $$

## Teacher Forcing

When the output is fedback $z_n = \hat{y}_n$, we can use what is called 'teacher forcing' to update the state.

This avoids BPTT, but can cause problems if trained in open loop, then deployed in closed loop mode.

Algorithm:
    
    z = 0
    for (x, y) in training set:
        y_hat = F(z, x, theta)
        error = y - y_hat
        theta = backprop(F, L, theta, error)
        z = y 
        

## RNN's, Signals and computation

Some observations:
- In the absence of input $x = 0$, the RNN obeys the 'Markov Propery' (next state only depends on the current state) and hence generates a stationary process $y_1,y_2,\ldots y_n$.
- Hence, RNN are causal (but there are some ways around this)!
- In the presence of input the process $y_1,y_2,\ldots$ is no longer stationary (courtesy of the graph $G$) and hence may have long-range dependence encoded in the interal state $z$
- RNN's are Turing complete (see Siegelmann and Sontag https://doi.org/10.1006/jcss.1995.1013 )

## Summary

- Recurrent Neural Networks (RNN's) are neural networks with a feedback loop. 
- RNN's are good for sequential data, and NLP in particular.
- RNN's have analogies in discrete nonlinear contorl systems
- Training a RNN usually involves Backpropogation through time, by training on fixed length sequences.
- This comes at the expense of parallelisation and may get stuck in local minimum.

## Thanks:
- Prof. Jonathan Manton 
- Selwyn Gomes
- The machine Learning reading group at UniMelb

&nbsp;

<center> https://github.com/peter-cudmore/seminars/ </center>